importing librarys

In [1]:
import random
import csv
import numpy as np

Load the CSV file

In [2]:
def loadCsv(filename):
    lines = csv.reader(open(filename))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]  # Convert String to Float numbers
    return dataset

In [3]:
def splitDataset(dataset,splitRatio):
    trainSize=int(len(dataset)*splitRatio)
    trainSet=[]
    copy=list(dataset)
    while len(trainSet) < trainSize:
        index=random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet,copy]

Split training set by class value

In [4]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

Find the mean and standard deviation of each feature in dataset

In [5]:
def summarize(dataset):
    summaries = [(np.mean(attribute), np.std(attribute)) for attribute in zip(*dataset)]
    del summaries[-1] #Remove last entry because it is class value.
    return summaries

find the mean and standard deviation of each feature in dataset by their class

In [6]:
def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

Calculate probability using gaussian density function

In [7]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return 1 / (math.sqrt(2 * math.pi) * stdev) * exponent

Calculate the class probability for input sample. Combine probability of each feature

In [8]:
def calculateClassProbabilities(summarizes, inputVector):
    probabilities = {}
    for classValue, classSummarizes in summarizes.items():
        probabilities[classValue] = 1
        for i in range(len(classSummarizes)):
            mean, stdev = classSummarizes[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

Compare probability for each class. Return the class label which has max probability

In [9]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None,-1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

Get class label for each value in test set

In [10]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

Create a naive bayes model. Then test the model and returns the testing result

In [11]:
def getAccuracy(testSet,predictions):
    correct=0
    for x in range(len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct +=1
    return (correct/float(len(testSet)))*100.0

In [15]:
def main():
    # load and prepare data
    filename ='.\\banknote.csv'
    splitRatio=0.67
    dataset=loadCsv(filename)
    trainingSet,testSet=splitDataset(dataset,splitRatio)
    print('split {0} rows into train = {1} and test = {2} rows '.format(len(dataset),len(trainingSet),len(testSet)))
    #prepare model
    summaries = summarizeByClass(trainingSet)
    #test model
    predictions=getPredictions(summaries,testSet)
    accuracy=getAccuracy(testSet,predictions)
    print('Accuracy:  {0}%'.format(accuracy))

In [23]:
main()

split 1372 rows into train = 919 and test = 453 rows 
Accuracy:  86.75496688741721%
